In [1]:
#Importing the necessary libraries.
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.externals import joblib
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

C:\Users\ssvin\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
#Reading the data from the csv

data = pd.read_csv('covid_data_production.csv')
data = data.drop('id',axis=1)
data = data.fillna(np.nan,axis=0)

In [3]:
#Fetching the encoder object from the development model
filename='ModelEncoding_object.sav'
EncodeModel = joblib.load(filename)

In [4]:
#Encoding the categorical data.
data['location'] = EncodeModel.fit_transform(data['location'].astype(str))
data['country'] = EncodeModel.fit_transform(data['country'].astype(str))
data['gender'] = EncodeModel.fit_transform(data['gender'].astype(str))
data[['symptom1']] = EncodeModel.fit_transform(data['symptom1'].astype(str))
data[['symptom2']] = EncodeModel.fit_transform(data['symptom2'].astype(str))
data[['symptom3']] = EncodeModel.fit_transform(data['symptom3'].astype(str))
data[['symptom4']] = EncodeModel.fit_transform(data['symptom4'].astype(str))
data[['symptom5']] = EncodeModel.fit_transform(data['symptom5'].astype(str))
data[['symptom6']] = EncodeModel.fit_transform(data['symptom6'].astype(str))

In [5]:
#Get the no of days taken from symptoms observed to the hospital visit

data['sym_on'] = pd.to_datetime(data['sym_on'])
data['hosp_vis'] = pd.to_datetime(data['hosp_vis'])
data['sym_on']= data['sym_on'].map(dt.datetime.toordinal)
data['hosp_vis']= data['hosp_vis'].map(dt.datetime.toordinal)
data['diff_sym_hos']= data['hosp_vis'] - data['sym_on']

In [6]:
data = data.drop(['sym_on','hosp_vis'],axis=1)

In [7]:
#Isnull check
data['age'] = data['age'].fillna(value=data['age'].mean())
for column in ['from_wuhan','vis_wuhan']:
    data[column].fillna(data[column].mode()[0], inplace=True)

In [8]:
data.isna().sum()

location          0
country           0
gender            0
age               0
vis_wuhan         0
from_wuhan        0
death           672
recov           672
symptom1          0
symptom2          0
symptom3          0
symptom4          0
symptom5          0
symptom6          0
diff_sym_hos      0
dtype: int64

In [9]:
#Fetching the scaling object from the development model
filename='ModelScaling_object.sav'
ScalingModel = joblib.load(filename)

In [10]:
X = data[['location','country','gender','age','vis_wuhan','from_wuhan','symptom1','symptom2','symptom3','symptom4','symptom5','symptom6','diff_sym_hos']]

In [11]:
#Scaling the input data
scaled_X = ScalingModel.fit(X).transform(X)

In [12]:
#Fetching the model object from the development model
filename='ModelClassification_object.sav'
Model = joblib.load(filename)

In [13]:
Y_Predict = Model.predict(scaled_X)

In [14]:
Y_Predict



array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
 Z =pd.DataFrame(Y_Predict)
Z['Mortality'] = Z

In [16]:
Z['Mortality']

0      0
1      0
2      0
3      1
4      0
      ..
953    0
954    0
955    0
956    0
957    0
Name: Mortality, Length: 958, dtype: int64

In [17]:
actualdata = pd.read_csv('covid_data_production.csv')

In [18]:
FinalResult = pd.concat([actualdata,Z[['Mortality']]],axis=1)

In [19]:
FinalResult

,id,location,country,gender,age,sym_on,hosp_vis,vis_wuhan,from_wuhan,death,recov,symptom1,symptom2,symptom3,symptom4,symptom5,symptom6,Mortality
0,800.0,Bavaria,Germany,male,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,801.0,Bavaria,Germany,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,802.0,Bavaria,Germany,male,52.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,803.0,Bavaria,Germany,male,33.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,804.0,Frankfurt,Germany,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [20]:
FinalResult.to_csv('Output.csv')